In [ ]:
!pip list

## Import Library

In [1]:
import pandas as pd
from IPython.display import display
import numpy as np

## Data Ingestion

In [231]:
df_admin = pd.read_csv('data_source/admin.csv', skiprows=4, names=["admin_id", "username", "name", "email", "password", "created_at", "updated_at", "deleted_at"])
df_challenge_confirmations = pd.read_csv('data_source/challenge_confirmations.csv', skiprows=4, names=["challenge_confirmations_id", "challenge_id", "user_id", "description", "date_upload", "challenge_confirmations_status", "created_at", "updated_at", "deleted_at"])
df_challenge_logs = pd.read_csv('data_source/challenge_logs.csv', skiprows=4,  names=["challenge_logs_id", "challenge_id", "user_id", "challenge_logs_status", "created_at", "updated_at", "deleted_at"])
df_challenges = pd.read_csv('data_source/challenges.csv', skiprows=4, names=["challenge_id", "author", "title", "difficulty", "description", "exp", "coin", "image_url", "date_start", "date_end", "impact_categories", "created_at", "updated_at", "deleted_at"])
df_impact_categories = pd.read_csv('data_source/impact_categories.csv', names=["impact_category_id", "name", "impact_poin", "icon_url", "created_at", "updated_at", "deleted_at"])
df_users = pd.read_csv('data_source/users.csv', skiprows=4, names=["user_id", "username", "password", "name", "email", "address", "gender", "phone", "exp", "coin", "avatar_url", "created_at", "deleted_at"])
df_challenge_impact = pd.read_csv('data_source/challenge_impact.csv', skiprows=4, names=["challenge_id", "impact_category_id", "created_at", "updated_at", "deleted_at"])
df_list = [df_admin, df_challenge_confirmations, df_challenge_logs, df_challenges, df_impact_categories, df_users, df_challenge_impact]

### Data Cleaning

In [ ]:
for df in df_list:
    df.info()

In [232]:
df_challenges = df_challenges.drop(columns="impact_categories")

In [233]:
df_ft_challenges = df_challenges[["challenge_id", "author"]]
df_ft_admin = df_admin[["admin_id"]]
df_ft_challenge_logs = df_challenge_logs[["challenge_logs_id", "challenge_id", "user_id", "challenge_logs_status"]]
df_ft_challenge_confirmations = df_challenge_confirmations[["challenge_confirmations_id", "challenge_id", "user_id", "challenge_confirmations_status", "date_upload"]]
df_ft_users = df_users[["user_id"]]
df_ft_challenge_impact = df_challenge_impact[["challenge_id", "impact_category_id"]]
df_ft_impact_categories = df_impact_categories[["impact_category_id", "impact_poin", "name"]]
# df_ft_list = [df_ft_admin, df_ft_challenge_confirmations, df_ft_challenge_logs, df_ft_challenges, df_ft_users, df_ft_challenge_impact, df_ft_impact_categories]

In [53]:
for df in df_ft_list:
    df.head()

## Data Transformation

### Join Fact Table

In [234]:
challenge_facts = pd.merge(df_ft_challenge_logs, df_ft_challenge_confirmations, on=["challenge_id", "user_id"], how='left')
challenge_facts = pd.merge(challenge_facts, df_ft_challenges, on="challenge_id", how='left')  
challenge_facts = pd.merge(challenge_facts, df_ft_challenge_impact, on="challenge_id", how='left')
challenge_facts = pd.merge(challenge_facts, df_ft_impact_categories, on="impact_category_id", how='left')    
challenge_facts = challenge_facts.drop(columns=["challenge_logs_id", "challenge_confirmations_id", "impact_category_id"])

In [226]:
challenge_facts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   challenge_id                    50 non-null     object
 1   user_id                         50 non-null     object
 2   challenge_logs_status           50 non-null     object
 3   challenge_confirmations_status  36 non-null     object
 4   date_upload                     36 non-null     object
 5   author                          50 non-null     object
 6   impact_poin                     50 non-null     object
 7   name                            50 non-null     object
dtypes: object(8)
memory usage: 3.2+ KB


In [227]:
challenge_facts.nunique()

challenge_id                      15
user_id                           15
challenge_logs_status              2
challenge_confirmations_status     3
date_upload                       18
author                             5
impact_poin                        8
name                              15
dtype: int64

### Handle Missing Value

In [235]:
challenge_facts['challenge_confirmations_status'] = challenge_facts.apply(lambda row: 
    'Challenge Tidak Diambil' if row['challenge_logs_status'] == 'Ditolak' 
    else ('Challenge Belum Dilakukan' if row['challenge_logs_status'] == 'Diterima' and pd.isnull(row['challenge_confirmations_status']) else row['challenge_confirmations_status']), 
    axis=1)

challenge_facts['date_upload'] = challenge_facts.apply(lambda row: 
    pd.Timestamp('1945-08-17') if (row['challenge_logs_status'] == 'Ditolak') or (row['challenge_logs_status'] == 'Diterima' and pd.isnull(row['date_upload'])) 
    else row['date_upload'], 
    axis=1)

In [236]:
challenge_facts

,challenge_id,user_id,challenge_logs_status,challenge_confirmations_status,date_upload,author,impact_poin,name
0,5fe8f4f6-43ee-4420-8969-bf1a0e19260c,5dcdb121-85df-487c-8d4a-b4fd5033c9c0,Diterima,Disetujui,2024-06-15 3:21:00,b1e7fdc4-6465-4f06-8914-85b36255368e,50,Berkelanjutan
1,5fe8f4f6-43ee-4420-8969-bf1a0e19260c,5dcdb121-85df-487c-8d4a-b4fd5033c9c0,Diterima,Disetujui,2024-06-15 3:21:00,b1e7fdc4-6465-4f06-8914-85b36255368e,35,Dapat Terurai
2,ba2621ac-6f99-43cb-ab8a-7db626b7c4e4,5b9a23e5-e562-4fae-b11c-787e655a67d7,Diterima,Disetujui,2024-07-05 12:30:33,cf98c726-fb72-4687-9c88-e8ce671c32d6,70,Tanpa Limbah
3,ba2621ac-6f99-43cb-ab8a-7db626b7c4e4,5b9a23e5-e562-4fae-b11c-787e655a67d7,Diterima,Disetujui,2024-07-05 12:30:33,cf98c726-fb72-4687-9c88-e8ce671c32d6,30,Dapat Didaur Ulang
4,5086c59a-e2f9-44d0-bcc6-f7fdb58f07a4,d4a59d1b-38fa-4736-88d7-00cdc02ef5be,Diterima,Disetujui,2024-08-17 5:21:40,06db6d61-e7c8-4126-ab0f-f2b6bc1458d1,40,Ramah Lingkungan
5,5086c59a-e2f9-44d0-bcc6-f7fdb58f07a4,d4a59d1b-38fa-4736-88d7-00cdc02ef5be,Diterima,Disetujui,2024-08-17 5:21:40,06db6d61-e7c8-4126-ab0f-f2b6bc1458d1,30,Dapat Didaur Ulang
6,aa013dc7-4fb3-4cf1-9cc0-e1a0ad60414d,6932e20c-3c55-4d95-91c5-ce316cc5843f,Ditolak,Challenge Tidak Diambil,1945-08-17 00:00:00,6a53c2db-2fa5-47df-8462-baea4308d48f,60,Rendah Karbon
7,aa013dc7-4fb3-4cf1-9cc0-e1a0ad60414d,6932e20c-3c55-4d95-91c5-ce316cc5843f,Ditolak,Challenge Tidak Diambil,1945-08-17 00:00:00,6a53c2db-2fa5-47df-8462-baea4308d48f,55,Efisien Energi
8,5550e03b-15c6-43c9-997c-4086adc8b573,8f25169f-5084-45c3-a99f-40daf59485d4,Ditolak,Challenge Tidak Diambil,1945-08-17 00:00:00,90aae29b-bf80-44bd-ab59-e7388e84e27f,55,Efisien Energi
9,5550e03b-15c6-43c9-997c-4086adc8b573,8f25169f-5084-45c3-a99f-40daf59485d4,Ditolak,Challenge Tidak Diambil,1945-08-17 00:00:00,90aae29b-bf80-44bd-ab59-e7388e84e27f,50,Hemat Air


## Data Load

In [237]:
# Read .env
from dotenv import load_dotenv
import os

# Upload to firebase
import firebase_admin
from firebase_admin import credentials
from firebase_admin import storage

In [238]:
def load_data(df):

    load_dotenv()
    CERTIFICATE_PATH = os.getenv('CERTIFICATE_PATH')
    GOOGLE_STORAGE_BUCKET = os.getenv('GOOGLE_STORAGE_BUCKET')

    try:
        cred = credentials.Certificate(CERTIFICATE_PATH)

        storage_bucket = GOOGLE_STORAGE_BUCKET

        firebase_admin.initialize_app(cred, {"storageBucket": storage_bucket})

        bucket = storage.bucket()

        filename = "challenge_fact_table.csv"

        df.to_csv(filename)

        blob = bucket.blob(blob_name=filename)

        blob.upload_from_filename(filename)

        print("Data loaded successfully")
            
    except Exception as e:
        print(f"An error occurred when loading data to the storage: {e}")

load_data(challenge_facts)

Data loaded successfully


In [99]:
df_logs = pd.read_csv('data_source/logs.csv')

In [100]:
df_logs.head()
df_logs.nunique()

insertId                              500
labels."commit-sha"                     3
labels."gcb-build-id"                   3
labels."gcb-trigger-id"                 3
labels."gcb-trigger-region"             1
labels.instanceId                       3
labels."managed-by"                     1
logName                                 2
receiveLocation                         0
receiveTimestamp                      149
resource.labels.configuration_name      3
resource.labels.location                2
resource.labels.project_id              1
resource.labels.revision_name           3
resource.labels.service_name            3
resource.type                           1
textPayload                           368
timestamp                             500
dtype: int64

In [106]:
with pd.option_context('display.max_rows', None):
    df_logs[["textPayload"]].head()